# Processing Column Data

Exploring functions available under `pyspark.sql.functions`

      1. Pre-defined Functions
      2. Create Dummy Data Frame
      3. Special Functions - col and lit
      4. String Manipulation Functions - 1
      5. String Manipulation Functions - 2
      6. Date and Time Arithmetic
      8. Date and Time - Extracting Information
      9. Using CASE and WHEN

### 1. Pre-defined Functions

In [0]:
# Reading data

orders = spark.read.csv('/public/retail_db/orders',schema = 'order_id int,order_date string,order_customer_id int,order_status string')
orders.show(5)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
+--------+--------------------+-----------------+---------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import *


In [0]:
orders.select('*',date_format('order_date','dd-MM-yy').alias('date')).show(2)
#OR
orders.withColumn('date',date_format('order_date','dd-MM-yy')).show(2)

+--------+--------------------+-----------------+---------------+--------+
|order_id|          order_date|order_customer_id|   order_status|    date|
+--------+--------------------+-----------------+---------------+--------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|25-07-13|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|25-07-13|
+--------+--------------------+-----------------+---------------+--------+
only showing top 2 rows

+--------+--------------------+-----------------+---------------+--------+
|order_id|          order_date|order_customer_id|   order_status|    date|
+--------+--------------------+-----------------+---------------+--------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|25-07-13|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|25-07-13|
+--------+--------------------+-----------------+---------------+--------+
only showing top 2 rows



In [0]:
orders.filter(date_format('order_date','dd-MM-yy') == '01-01-14').show(2)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|   25876|2014-01-01 00:00:...|             3414|PENDING_PAYMENT|
|   25877|2014-01-01 00:00:...|             5549|PENDING_PAYMENT|
+--------+--------------------+-----------------+---------------+
only showing top 2 rows



In [0]:
orders.groupBy(date_format('order_date','yyyy').alias('Date')).count().show()

+----+-----+
|Date|count|
+----+-----+
|2014|38221|
|2013|30662|
+----+-----+



###  2. Create Dummy Data Frame

In [0]:
# SELECT * from dual gives 'X'
# dual - dummy CHAR(1)

l = [('X', )]
df = spark.createDataFrame(l, "dummy STRING")

from pyspark.sql.functions import current_date
df.select(current_date()). \
    show()

+--------------+
|current_date()|
+--------------+
|    2022-12-15|
+--------------+



### 3. Special Functions - col and lit

In [0]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [0]:
from pyspark.sql.functions import *

employeesDF. \
    select(concat(col("first_name"), 
                  lit(", "), 
                  col("last_name")
                 ).alias("full_name")
          ). \
    show(truncate=False)

+------------+
|full_name   |
+------------+
|Scott, Tiger|
|Henry, Ford |
|Nick, Junior|
|Bill, Gomes |
+------------+



In [0]:
employeesDF. \
    select(upper(col("first_name")), upper(col("last_name"))). \
    show()

+-----------------+----------------+
|upper(first_name)|upper(last_name)|
+-----------------+----------------+
|            SCOTT|           TIGER|
|            HENRY|            FORD|
|             NICK|          JUNIOR|
|             BILL|           GOMES|
+-----------------+----------------+



### 4. String Manipulation Functions - 1

**``Tasks - Case Conversion and length``**

Let us perform tasks to understand the behavior of case conversion functions and length.

* Use employees data and create a Data Frame.
* Apply all 5 functions (lower, upper, initcap, concat,length) on **nationality** and see the results.

In [0]:
employeesDF. \
  select("employee_id", "nationality"). \
  withColumn("nationality_upper", upper(col("nationality"))). \
  withColumn("nationality_lower", lower(col("nationality"))). \
  withColumn("nationality_initcap", initcap(col("nationality"))). \
  withColumn("nationality_length", length(col("nationality"))). \
  withColumn("nationality_id", concat("employee_id",lit(" - "),"nationality")). \
  show()

+-----------+--------------+-----------------+-----------------+-------------------+------------------+------------------+
|employee_id|   nationality|nationality_upper|nationality_lower|nationality_initcap|nationality_length|    nationality_id|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+------------------+
|          1| united states|    UNITED STATES|    united states|      United States|                13| 1 - united states|
|          2|         India|            INDIA|            india|              India|                 5|         2 - India|
|          3|united KINGDOM|   UNITED KINGDOM|   united kingdom|     United Kingdom|                14|3 - united KINGDOM|
|          4|     AUSTRALIA|        AUSTRALIA|        australia|          Australia|                 9|     4 - AUSTRALIA|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+------------------+



> *Q: What is the difference between **`concat`** and **`concat_ws`** when comes to string manipulation?*

###5. String Manipulation Functions - 2

In [0]:
from pyspark.sql.functions import concat_ws  #Withing its argument it takes separator. No need to call lit(literals) separately.

employeesDF. \
  select("employee_id", "nationality"). \
  withColumn("nationality_upper", upper(col("nationality"))). \
  withColumn("nationality_lower", lower(col("nationality"))). \
  withColumn("nationality_initcap", initcap(col("nationality"))). \
  withColumn("nationality_length", length(col("nationality"))). \
  withColumn("nationality_id", concat_ws("-","employee_id","nationality")). \
  show()

+-----------+--------------+-----------------+-----------------+-------------------+------------------+----------------+
|employee_id|   nationality|nationality_upper|nationality_lower|nationality_initcap|nationality_length|  nationality_id|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+----------------+
|          1| united states|    UNITED STATES|    united states|      United States|                13| 1-united states|
|          2|         India|            INDIA|            india|              India|                 5|         2-India|
|          3|united KINGDOM|   UNITED KINGDOM|   united kingdom|     United Kingdom|                14|3-united KINGDOM|
|          4|     AUSTRALIA|        AUSTRALIA|        australia|          Australia|                 9|     4-AUSTRALIA|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+----------------+



In [0]:
from pyspark.sql.functions import split, explode, lit

l = [('X', )]
df = spark.createDataFrame(l, "dummy STRING")

df.select(split(lit("Hello World, how are you"), " ")).show(truncate=False)  #this converts to an array/list
df.select(explode(split(lit("Hello World, how are you"), " ")).alias('word')).show(truncate=False) # this converts split output to rows

+--------------------------------------+
|split(Hello World, how are you,  , -1)|
+--------------------------------------+
|[Hello, World,, how, are, you]        |
+--------------------------------------+

+------+
|word  |
+------+
|Hello |
|World,|
|how   |
|are   |
|you   |
+------+



###6. Date and Time Arithmetic

In [0]:
from pyspark.sql.functions import current_date, current_timestamp

l = [('X', )]
df = spark.createDataFrame(l, "dummy STRING")

df.select(current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2022-12-15|
+--------------+



In [0]:
df.select(current_timestamp()).show()
print('Now, if I want to see complete string')
df.select(current_timestamp()).show(truncate = False)

+--------------------+
| current_timestamp()|
+--------------------+
|2022-12-15 19:02:...|
+--------------------+

Now, if I want to see complete string
+-----------------------+
|current_timestamp()    |
+-----------------------+
|2022-12-15 19:02:43.583|
+-----------------------+



* We can convert a string which contain date or timestamp in non-standard format to standard date or time using `to_date` or `to_timestamp` function respectively.

In [0]:
df.select(to_date(lit('20210208'),'yyyymmdd')).alias('date').show()

+---------------------------+
|to_date(20210208, yyyymmdd)|
+---------------------------+
|                 2021-01-08|
+---------------------------+



**``Tasks``**
* Get help on each and every function first and understand what all arguments need to be passed.
* Create a Dataframe by name datetimesDF with columns date and time.

In [0]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]
datetimesDF = spark.createDataFrame(datetimes, schema="date STRING, time STRING")

In [0]:
datetimesDF. \
    withColumn("months_between_date", round(months_between(current_date(), "date"), 2)). \
    withColumn("months_between_time", round(months_between(current_timestamp(), "time"), 2)). \
    withColumn("add_months_date", add_months("date", 3)). \
    withColumn("add_months_time", add_months("time", 3)). \
    show(truncate=False)

+----------+-----------------------+-------------------+-------------------+---------------+---------------+
|date      |time                   |months_between_date|months_between_time|add_months_date|add_months_time|
+----------+-----------------------+-------------------+-------------------+---------------+---------------+
|2014-02-28|2014-02-28 10:00:00.123|105.58             |105.59             |2014-05-28     |2014-05-28     |
|2016-02-29|2016-02-29 08:08:08.999|81.55              |81.56              |2016-05-29     |2016-05-29     |
|2017-10-31|2017-12-31 11:59:59.123|61.48              |59.49              |2018-01-31     |2018-03-31     |
|2019-11-30|2019-08-31 00:00:00.000|36.52              |39.51              |2020-02-29     |2019-11-30     |
+----------+-----------------------+-------------------+-------------------+---------------+---------------+



### 8. Date and Time Extract Functions

In [0]:
df.select(
    current_date().alias('current_date'), 
    year(current_date()).alias('year'),
    month(current_date()).alias('month'),
    weekofyear(current_date()).alias('week of year'),
    dayofyear(current_date()).alias('day of year'),
    dayofmonth(current_date()).alias('day of month'),
    dayofweek(current_date()).alias('day of week')
).show() #yyyy-MM-dd

+------------+----+-----+------------+-----------+------------+-----------+
|current_date|year|month|week of year|day of year|day of month|day of week|
+------------+----+-----+------------+-----------+------------+-----------+
|  2022-12-15|2022|   12|          50|        349|          15|          5|
+------------+----+-----+------------+-----------+------------+-----------+



In [0]:
df.select(
    current_timestamp().alias('current_timestamp'), 
    year(current_timestamp()).alias('year'),
    month(current_timestamp()).alias('month'),
    dayofmonth(current_timestamp()).alias('day of month'),
    hour(current_timestamp()).alias('hour'),
    minute(current_timestamp()).alias('minute'),
    second(current_timestamp()).alias('second')
).show(truncate=False) #yyyy-MM-dd HH:mm:ss.SSS

+----------------------+----+-----+------------+----+------+------+
|current_timestamp     |year|month|day of month|hour|minute|second|
+----------------------+----+-----+------------+----+------+------+
|2022-12-15 19:02:45.09|2022|12   |15          |19  |2     |45    |
+----------------------+----+-----+------------+----+------+------+



###  9. Using CASE and WHEN

In [0]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [0]:
employeesDF. \
    withColumn(
        'bonus', 
        expr("""
            CASE WHEN bonus IS NULL OR bonus = '' THEN 0
            ELSE bonus
            END
            """)
    ). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|    0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|    0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+

